In [16]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from qiskit.circuit.library import Initialize

# Quantum Generative Adversarial Network

This notebook will aim to implement a quantum generator and classical discriminator to check the probability distribution of classical training samples.

Note: This will be based off of work provided in Nature from Zoufal et al. (2019)

The quantum generator is trained to transform an input state into an output state, based on the following,

\begin{equation}
G_{\theta} |\psi_{\text{in}}\rangle = g_{\theta} = \sum_{j = 0}^{2^n - 1} \sqrt{p_{\theta}^j} \, |j\rangle
\end{equation}

p is the occurrence probabilities of basis state j. The sample space X is from {0, ..., 2^n-1}.

The qGen is implemented in a variational form, a parametrised quantum circuit. The k layers consist of alternating rotation-Y block with entanglement blocks U_ent. The rotations are parametrised by theta^(i, j) for ith qubit in the jth layer. for n qubits the variational circuit uses (k+1)n parametrised single qubit gates, and kn two-qubits gates.

In [17]:
# Quantum Generator circuit
def gen(theta, n, k):
    """
        This function generates the quantum generator circuit for n qubits. It takes in
        parameters theta in an array format.
    """
    qc = QuantumCircuit(n)

    for j in range(k):
        # Rotation block
        for i in range(n):
            qc.ry(theta[i, j], i)

        # Entanglement block
        if j != k - 1:
            for i in range(n - 1):
                qc.cz(i, i+1)
            qc.cz(n - 1, 0)

    return qc

We now need to calculate the loss functions, given m data samples and g^l from the quantum generator. With m randomly chosen training data samples x^l for l = 1, ..., m the loss functions are:

For the generator:
\begin{equation}
L_G (\phi, \theta) = - \frac{1}{m} \sum^m_{l=1} \left[ log D_{\phi} (g^l) \right]
\end{equation}

For the discriminator (standard binary cross-entropy loss):
\begin{equation}
L_D (\phi, \theta) = - \frac{1}{m} \sum^m_{l=1} \left[ log D_{\phi} (x^l) + log(1 - D_{\phi} (g^l)) \right]
\end{equation}

## Generator input state preparation
We create a normal distribution with $\mu$ and $\sigma$ being the estimates of mean and standard deviation of training data samples. So we need to figure out how to create a normal distribution for the input state.

In [18]:
# This is included in the finance functions
from qiskit_finance.circuit.library.probability_distributions import NormalDistribution as ND

def norm_distr(n, mu, sigma, bounds):
    """
        Generates a normal distribution based on the parameters provided.
    """
    qc = ND(n, mu=mu, sigma=sigma, bounds=bounds)
    return qc

The thing to know with this is that the normaldistribution is not efficiently found for a quantum state. So this function estimates the likely normal distribution based on the parameters. We're basically abusing the difficulty in generation by using prebuilt functions in qiskit, but it's not too bad otherwise, just more code.

## Discriminator
This is the classical implementation of our qGAN, it will consist of a 50-node input layer, a 20-node hidden layer, and a single-node output layer. We'll use linear transforms followed by leaky ReLUs (maybe GELU depending on whether it works or not), then a linear transform at the output layer and a sigmoid function.

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [20]:
class discriminator(nn.Module):
    def __init__(self, input_size=50, hidden_size=20, output_size=1):
        super(discriminator, self).__init__()

        # First linear layer
        self.l1 = nn.Linear(input_size, hidden_size)

        # Leaky relu
        self.leaky_relu = nn.LeakyReLU(0.01)

        # Second linear layer
        self.l2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # First linear layer
        x = self.l1(x)
        
        # Apply leaky ReLU
        x = self.leaky_relu(self.l1(x))

        # Apply second layer
        x = self.l2(x)

        # Sigmoid activation
        x = torch.sigmoid(x)

        return x

In [21]:
# We can also just use a sequential approach given it's probably easier
model = nn.Sequential(
    nn.Linear(50, 20),
    nn.LeakyReLU(0.01),
    nn.Linear(20, 1),
    nn.Sigmoid()
)

## Trainer
The qGAN is trained using AMSGrad with an inital learning rate of 10^-4. The stability of the training is improved with a gradient penalty on the discriminator's loss function.

Training data is split into batches of 2000.

The generated data samples are created by preparing and measuring the qGen 2000 times, the batches are then used to update the parameters of the discriminator and the generator in an alternating fashion.

In [22]:
# Define the number of parameters (n x k)
n = 4
k = 8

# Generator parameters
gen_params = nn.Parameter(torch.randn(n, k, requires_grad=True))

# Generator optimiser
gen_opt = optim.Adam([gen_params], lr=10**(-4), amsgrad=True)

# Discriminator optimiser
dis_opt = optim.Adam(model.parameters(), lr=10**(-4), amsgrad=True)

In [23]:
def train_qgan(
    quantum_generator_fn, generator_params, discriminator, dataloader, num_epochs=100,
    disc_lr=1e-4, latent_dim=8, device='cpu', generator_update_fn=None
):

    discriminator.to(device)
    epsilon = 1e-8

    dis_opt = optim.Adam(discriminator.parameters(), lr=disc_lr, amsgrad=True)

    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}")
        for real_data_batch in tqdm(dataloader, desc="Training step"):
            real_data = real_data_batch[0].to(device)
            batch_size = real_data.size(0)

            # === Train Discriminator ===
            dis_opt.zero_grad()

            # Real data
            real_preds = discriminator(real_data)
            real_loss = torch.log(real_preds + epsilon)

            # Fake data (from quantum circuit)
            noise = torch.randn(batch_size, latent_dim)
            fake_data = quantum_generator_fn(noise, generator_params).detach().to(device)
            fake_preds = discriminator(fake_data)
            fake_loss = torch.log(1 - fake_preds + epsilon)

            disc_loss = -torch.mean(real_loss + fake_loss)
            disc_loss.backward()
            dis_opt.step()

            # === Train Generator ===
            # Do NOT use .backward() — assume manual gradient update
            noise = torch.randn(batch_size, latent_dim)
            fake_data = quantum_generator_fn(noise, generator_params).to(device)
            fake_preds = discriminator(fake_data)

            gen_loss = -torch.mean(torch.log(fake_preds + epsilon))

            generator_update_fn(generator_params, gen_loss, noise, discriminator)

        print(f"Epoch [{epoch+1}/{num_epochs}]  D_loss: {disc_loss.item():.4f}  G_loss: {gen_loss.item():.4f}")

### Updating the qGenerator
Our quantum generator is insufficient in its current capacity for the training cycles, so we need to update it based on what our training function does.

#### Adding an extra bit in here to allow for a normal encoding that doesn't blow up in size

In [24]:
def custom_normal_encoding(n, mu=0, sigma=1, bounds=(-np.pi, np.pi)):
    dim = 2 ** n
    x = np.linspace(bounds[0], bounds[1], dim)

    # Evaluate normal distribution (PDF)
    probs = np.exp(-0.5 * ((x - mu) / sigma) ** 2)
    probs /= np.sum(probs)  # Normalize

    # Convert to amplitudes
    amps = np.sqrt(probs)

    # Create quantum circuit
    qc = QuantumCircuit(n)
    init = Initialize(amps)
    qc.append(init, range(n))

    return qc

In [25]:
from qiskit_aer.primitives import Sampler

def quantum_gen_fn(noise_batch, theta, n, k, bounds, mu, sigma):
    sampler = Sampler()
    circuits = []

    for noise_vec in noise_batch:
        # Convert theta to np
        theta_np = theta.detach().cpu().numpy()

        # Build quantum circuit with noise encoding
        qc = QuantumCircuit(n)
        qc = custom_normal_encoding(n, mu=mu, sigma=sigma, bounds=bounds)

        # Apply rotation + entanglement blocks
        for j in range(k):
            for i in range(n):
                qc.ry(theta_np[i, j], i)
            if j != k - 1:
                for i in range(n - 1):
                    qc.cz(i, i+1)
                qc.cz(n - 1, 0)

        qc.measure_all()
        circuits.append(qc)

    results = sampler.run(circuits).result()

    # Extract probabilities
    batch_outputs = []
    for i, result in enumerate(results.quasi_dists):
        probs = np.zeros(n)
        for bitstring, prob in result.items():
            bits = [int(b) for b in format(int(bitstring), f'0{n}b')]
            probs += np.array(bits) * prob
        batch_outputs.append(probs)

    return torch.tensor(batch_outputs, dtype=torch.float32)

# Gradient update function
Now we want the function that updates the gradient for the generator. In the supplementary materials of the paper, they detailed how they update parameters, this is pretty complex so we'll just accept it and hope it works out.

In [26]:
def parameter_shift_update(theta, noise_batch, discriminator, lr, n_qubits, k_layers, bounds, mu, sigma):
    n, k = theta.shape
    batch_size = noise_batch.size(0)
    shift = np.pi / 2

    grad = torch.zeros_like(theta)

    with torch.no_grad():
        for i in range(n):
            for j in range(k):
                # Shifted parameter sets
                theta_plus = theta.clone()
                theta_plus[i, j] += shift
                theta_minus = theta.clone()
                theta_minus[i, j] -= shift

                # Generate samples using shifted parameters
                g_plus = quantum_gen_fn(noise_batch, theta_plus, n_qubits, k_layers, bounds, mu, sigma)
                g_minus = quantum_gen_fn(noise_batch, theta_minus, n_qubits, k_layers, bounds, mu, sigma)

                # Evaluate discriminator outputs
                D_plus = discriminator(g_plus)
                D_minus = discriminator(g_minus)

                # Compute gradient: average over batch
                log_d_plus = torch.log(D_plus + 1e-8)
                log_d_minus = torch.log(D_minus + 1e-8)
                grad[i, j] = -torch.mean((log_d_plus - log_d_minus) / 2)

        # Gradient descent step
        theta -= lr * grad

## Loading the data
Because I haven't gotten a database of information we want, we'll ignore that detail for the time being and search later on for something to apply this to. Our data will have batches of 2000 items, so we'll use that to load the data

In [27]:
from torch.utils.data import TensorDataset, DataLoader

def train_loader(real_data=None, num_samples = 2000, feature_dim=50, batch_size=32, shuffle=True):
    if real_data is None:
        real_data = torch.randn(num_samples, feature_dim)

    dataset = TensorDataset(real_data)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
    return loader

In [28]:
import numpy as np
from qiskit import QuantumCircuit
from tqdm import tqdm

mu = 0.5
sigma = 1
bounds = [-np.pi, np.pi]
n = 6
k = 1
theta = nn.Parameter(torch.randn(n, k, dtype=torch.float32, requires_grad=True))

model = nn.Sequential(
    nn.Linear(n, 20),
    nn.LeakyReLU(0.01),
    nn.Linear(20, 1),
    nn.Sigmoid()
)

train_qgan(
    quantum_generator_fn=lambda noise, theta: quantum_gen_fn(
        noise, theta, n=n, k=k, bounds=bounds, mu=mu, sigma=sigma
    ),
    generator_params=theta,  # theta is a torch.nn.Parameter with shape (n, k)
    discriminator=model,
    dataloader=train_loader(num_samples=1000, feature_dim=n, batch_size=32),
    generator_update_fn=lambda theta, loss, noise, disc: parameter_shift_update(
        theta, noise, disc, lr=0.01, n_qubits=n, k_layers=k, bounds=bounds, mu=mu, sigma=sigma
    )
)

Epoch 1


Training step:   0%|                                     | 0/32 [00:00<?, ?it/s]/var/folders/9v/3k54msls5_lcsl4_4qdlbhn40000gn/T/ipykernel_26354/3927006739.py:38: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:257.)
  return torch.tensor(batch_outputs, dtype=torch.float32)
Training step: 100%|████████████████████████████| 32/32 [00:30<00:00,  1.06it/s]


Epoch [1/100]  D_loss: 1.4039  G_loss: 0.8008
Epoch 2


Training step: 100%|████████████████████████████| 32/32 [00:30<00:00,  1.05it/s]


Epoch [2/100]  D_loss: 1.3809  G_loss: 0.8005
Epoch 3


...

Epoch [67/100]  D_loss: 0.7689  G_loss: 1.0736
Epoch 68


Training step: 100%|████████████████████████████| 32/32 [01:12<00:00,  2.25s/it]


Epoch [68/100]  D_loss: 0.7533  G_loss: 1.0800
Epoch 69


Training step:  81%|██████████████████████▊     | 26/32 [01:06<00:15,  2.56s/it]


KeyboardInterrupt: 

### Errors encountered:
Typos, qubit count too large (50 blew up the data to 8 PiB!!! o.o), needed to see progress, so adding a loading bar with tqdm

#### More information on the data explosion!
So basically with 50 qubits, and the encoding of the normal distribution, our data blows up by 2^50 points on a grid, which is 8 petabytes of memory for float64 values, an absolutely crazy amount. We might modify the normal distribution to allow ourself to still have the normal distribution but better fitted.